## Import dependencies

In [0]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

## Setup notebook parameters

In [0]:
# Train parameters
INIT_LEARNING_RATE = 1e-4
EPOCHS             = 20
BATCH_SIZE         = 32

# Data parameters
root_path  = '/content/drive/My Drive/covid-19/'
data_path  = root_path + 'dataset/'
model_path = root_path + 'detection_mask_model'

## Load data

In [3]:
def load_image_paths():
  file_paths_and_labels = []
  subfolders            = [x[0] for x in os.walk(data_path)][1:]
  
  for folder in subfolders:
    label = folder.split('/')[-1]
    
    [file_paths_and_labels.append((folder + '/' + file_name, label)) for file_name in os.listdir(folder)]

  return file_paths_and_labels

def load_images():
  file_paths_and_labels = load_image_paths()
  data   = []
  labels = []
  
  total_images = len(file_paths_and_labels)

  for i, (image_path, label) in enumerate(file_paths_and_labels):
    print("Loading images: %.2f%%" % (100*(i+1)/total_images))

    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(label)
  
  data   = np.array(data, dtype="float32")
  labels = np.array(labels)
  
  return data, labels

data, labels = load_images()

Loading images: 0.07%
Loading images: 0.14%
Loading images: 0.22%
Loading images: 0.29%
Loading images: 0.36%
Loading images: 0.43%
Loading images: 0.51%
Loading images: 0.58%
Loading images: 0.65%
Loading images: 0.72%
Loading images: 0.79%
Loading images: 0.87%
Loading images: 0.94%
Loading images: 1.01%
Loading images: 1.08%
Loading images: 1.16%
Loading images: 1.23%
Loading images: 1.30%
Loading images: 1.37%
Loading images: 1.44%
Loading images: 1.52%
Loading images: 1.59%
Loading images: 1.66%
Loading images: 1.73%
Loading images: 1.81%
Loading images: 1.88%
Loading images: 1.95%
Loading images: 2.02%
Loading images: 2.09%
Loading images: 2.17%
Loading images: 2.24%
Loading images: 2.31%
Loading images: 2.38%
Loading images: 2.45%
Loading images: 2.53%
Loading images: 2.60%
Loading images: 2.67%
Loading images: 2.74%
Loading images: 2.82%
Loading images: 2.89%
Loading images: 2.96%
Loading images: 3.03%
Loading images: 3.10%
Loading images: 3.18%
Loading images: 3.25%
Loading im

## One-Hot encoding, Data split (train/test) and Data augmentation

In [0]:
label_binarizer = LabelBinarizer()
labels          = to_categorical(label_binarizer.fit_transform(labels))

(trainX, testX, trainY, testY) = train_test_split(data, 
                                                  labels, 
																								  test_size		 = 0.20, 
																									stratify		 = labels, 
																									random_state = 7)

augmentation_pipeline = ImageDataGenerator(rotation_range 		= 20,
												 								 	 zoom_range 				= 0.15,
												 								 	 width_shift_range 	= 0.2,
												 								 	 height_shift_range = 0.2,
												 								 	 shear_range 				= 0.15,
												 								 	 horizontal_flip 		= True,
												 								 	 fill_mode 					= "nearest")

## Load Pre-trained model and implement the custom model layers

In [5]:
def get_model():
	head_model = MobileNetV2(weights     = "imagenet", 
													include_top  = False, 
													input_tensor = Input(shape=(224, 224, 3)))

	base_model = head_model.output
	base_model = AveragePooling2D(pool_size=(7, 7))(base_model)
	base_model = Flatten(name="flatten")(base_model)
	base_model = Dense(128, activation="relu")(base_model)
	base_model = Dropout(0.5)(base_model)
	base_model = Dense(2, activation="softmax")(base_model)

	model = Model(inputs=head_model.input, outputs=base_model)

	# Freeze pre-trained model weights
	for layer in head_model.layers:
		layer.trainable = False

	optimizer = Adam(lr   = INIT_LEARNING_RATE, 
									decay = INIT_LEARNING_RATE / EPOCHS)

	model.compile(loss 			= "binary_crossentropy", 
								optimizer = optimizer, 
								metrics 	= ["accuracy"])

	return model

model = get_model()

## Training

In [12]:
augmented_training_data = augmentation_pipeline.flow(trainX, trainY, batch_size=BATCH_SIZE)

training_history = model.fit(augmented_training_data,
														 steps_per_epoch  = len(trainX) // BATCH_SIZE,
														 epochs 					= EPOCHS)

Epoch 1/20
34/34 [==============================] - 12s 344ms/step - loss: 0.0095 - accuracy: 0.9972
Epoch 2/20
34/34 [==============================] - 12s 347ms/step - loss: 0.0097 - accuracy: 0.9963
Epoch 3/20
34/34 [==============================] - 12s 341ms/step - loss: 0.0103 - accuracy: 0.9972
Epoch 4/20
34/34 [==============================] - 12s 341ms/step - loss: 0.0048 - accuracy: 0.9991
Epoch 5/20
34/34 [==============================] - 12s 346ms/step - loss: 0.0094 - accuracy: 0.9963
Epoch 6/20
34/34 [==============================] - 12s 345ms/step - loss: 0.0086 - accuracy: 0.9972
Epoch 7/20
34/34 [==============================] - 12s 345ms/step - loss: 0.0080 - accuracy: 0.9991
Epoch 8/20
34/34 [==============================] - 12s 345ms/step - loss: 0.0101 - accuracy: 0.9981
Epoch 9/20
34/34 [==============================] - 12s 343ms/step - loss: 0.0047 - accuracy: 0.9991
Epoch 10/20
34/34 [==============================] - 12s 344ms/step - loss: 0.0148 - accura

## Model test report

In [13]:
predicted_classes = model.predict(testX, batch_size=BATCH_SIZE)
predicted_classes = np.argmax(predicted_classes, axis=1)

print(classification_report(testY.argmax(axis=1), 
                            predicted_classes,
														target_names = label_binarizer.classes_))

              precision    recall  f1-score   support

   with_mask       1.00      0.99      1.00       138
without_mask       0.99      1.00      1.00       139

    accuracy                           1.00       277
   macro avg       1.00      1.00      1.00       277
weighted avg       1.00      1.00      1.00       277



Model saving

In [0]:
model.save(model_path, save_format="h5")